In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
import jax
import jax_flock as jf
import functools

params = jf.Params(
    weight_forward  = 0.20,
    weight_separate = 1.00,
    weight_align    = 0.30,
    weight_cohere   = 0.60,
    weight_avoid    = 0.80
)

config = jf.Config(
    boid_count = 100,
    sphere_radius = 30.0
)

time_step = 1 / 60

next_state = functools.partial(jf.next_state, params, config, time_step)
next_state = jax.jit(next_state)

state = jf.init_state(config, seed=0)
state = next_state(state)


In [42]:

%timeit next_state(state)

1.9 ms ± 5.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
